In [1]:
#imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import multiprocessing
import gensim
from gensim.utils import simple_preprocess
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from langdetect import detect
# from wordcloud import WordCloud, STOPWORDS
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import svm
# from better_profanity import profanity
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.linear_model import Ridge #import ridge 

#imports
%matplotlib inline
import numpy as np 
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [2]:
veclyrics = np.load('veclyrics.pkl',allow_pickle = True)
df = pd.read_pickle("response.pkl")
X_train_one, X_test_one, y_train_one, y_test_one = train_test_split(veclyrics, df['pop'], test_size = 0.2, random_state = 1)
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(veclyrics, df.iloc[:,3:10], test_size = 0.2, random_state = 1)

In [3]:
#doesn't take n_jobs?
#also doesn't want to take multilabel by default
gb = GradientBoostingClassifier(verbose = True)
gb.fit(X_train_one, y_train_one)

      Iter       Train Loss   Remaining Time 
         1           1.3699            3.98m
         2           1.3654            3.90m
         3           1.3614            3.82m
         4           1.3580            3.73m
         5           1.3549            3.68m
         6           1.3518            3.62m
         7           1.3492            3.58m
         8           1.3468            3.52m
         9           1.3444            3.48m
        10           1.3421            3.42m
        20           1.3248            3.10m
        30           1.3116            2.78m
        40           1.3014            2.37m
        50           1.2929            1.98m
        60           1.2856            1.58m
        70           1.2786            1.19m
        80           1.2722           47.49s
        90           1.2663           23.82s
       100           1.2608            0.00s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=True,
                           warm_start=False)

In [5]:
#gb is slow and, for pop at least, way less accurate with default parameters than rf.  
#also, it doesn't easily take multilabel.
gb.score(X_test_one,y_test_one)

0.6227263837277528

In [6]:
rf = RandomForestClassifier(verbose = True, n_jobs= -1)
rf.fit(X_train_all, y_train_all)

C:\Users\peter\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    4.5s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.9s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=True,
                       warm_start=False)

In [51]:
#y_pred_rf vs y_test
y_pred_rf = rf.predict(X_test_all)

acc = []

for i in range(0,7):
    check = y_pred_rf[:,i] == np.array(y_test_all.iloc[:,i])
    accuracy = check.sum()/len(check)
    acc.append(accuracy)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


In [53]:
acc

[0.7667709759436729,
 0.8194797574809309,
 0.5722667709759437,
 0.8496968511637004,
 0.7910228828476432,
 0.8212399765304127,
 0.9355564247995306]

In [55]:
y_pred_rf

array([[1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]])

In [58]:
y_test_all.sum()/len(y_test_all)

rock                 0.751125
singer-songwriter    0.179542
pop                  0.450714
metal                0.151770
folk                 0.207412
country              0.180716
hip hop / rap        0.081459
dtype: float64

In [54]:
y_test_all

rock  singer-songwriter  pop  metal  folk  country  hip hop / rap
42285     1                  0    0      1     0        0              0
31291     1                  1    0      0     0        0              0
9853      1                  0    0      0     0        0              0
42247     1                  0    0      1     0        0              0
15061     1                  0    0      0     0        0              1
30276     0                  0    1      0     0        0              1
30069     1                  1    1      0     1        0              0
30578     0                  0    0      1     0        0              1
34208     0                  0    1      0     0        0              0
10882     1                  0    1      0     0        0              0
31088     1                  0    0      0     1        1              0
15412     1                  0    1      0     0        1              0
12830     1                  0    1      0     1        0              0
38547     1                  0    1      0     0        0              0
40729     0                  0    1      0     0        0              0
17711     1                  0    1      0     0        0              0
5748      1                  1    0      0     0        0              0
25775     1                  0    0      0     0        0              0
6734      0                  0    1      0     0        0              0
34043     1                  0    0      0     0        0              0
53026     1                  0    0      0     0        1              0
49699     0                  0    0      0     0        1              0
31315     1                  1    0      0     0        0              0
42553     1                  0    0      0     0        0              0
40815     0                  0    1      0     0        0              0
18246     1                  1    1      0     1        1              0
40281     1                  1    0      0     1        0              0
6616      0                  0    1      0     0        0              0
8192      1                  0    0      0     0        0              0
18059     0                  0    1      0     0        0              0
...     ...                ...  ...    ...   ...      ...            ...
54215     1                  0    1      0     0        0              0
54940     1                  0    0      0     1        0              0
49050     0                  0    1      0     0        0              0
33188     1                  0    0      0     0        0              0
52509     1                  1    0      0     1        0              0
9884      1                  0    0      0     0        0              0
8832      1                  0    0      0     0        1              0
13308     1                  1    1      0     1        0              0
7468      0                  0    1      0     0        0              1
55054     1                  0    0      0     0        1              0
57172     1                  0    0      1     0        0              1
8500      1                  0    1      0     0        0              0
50554     1                  1    0      0     1        1              0
24105     1                  1    0      0     1        1              0
49780     1                  0    1      0     0        0              0
7507      0                  0    1      0     0        0              1
50500     1                  0    0      0     0        0              0
20184     0                  0    1      0     0        0              0
36136     1                  1    1      0     1        0              0
52168     0                  0    1      0     0        0              1
50770     1                  0    0      0     0        0              0
46625     1                  0    1      0     1        0              0
17592     1                  0    0      0     0        1        

In [50]:
sum(acc)/len(acc)

0.7937190913916907

In [52]:
print('test report',classification_report(y_test_all, y_pred_rf))

test report               precision    recall  f1-score   support

           0       0.79      0.95      0.86      7681
           1       0.46      0.03      0.06      1836
           2       0.54      0.36      0.43      4609
           3       0.60      0.03      0.05      1552
           4       0.46      0.04      0.07      2121
           5       0.56      0.05      0.09      1848
           6       0.95      0.22      0.36       833

   micro avg       0.72      0.46      0.56     20480
   macro avg       0.62      0.24      0.28     20480
weighted avg       0.64      0.46      0.46     20480
 samples avg       0.73      0.52      0.56     20480



In [37]:
#appears to only be correct when whole vector of predictions is correct.  our metrics should check genre by genre.
rf.score(X_test_all,y_test_all)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


0.20144729121846275

In [ ]:
#why is classificaiton report giving different numbers from direct calcs?

In [40]:
#how i would normally check an individual response class
y_pred_rf = rf.predict(X_test_all)
y_pred_train_rf = rf.predict(X_train_all)
print('test report',classification_report(y_test_all, y_pred_rf))
print('train report',classification_report(y_train_all, y_pred_train_rf))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


test report               precision    recall  f1-score   support

           0       0.79      0.95      0.86      7681
           1       0.46      0.03      0.06      1836
           2       0.54      0.36      0.43      4609
           3       0.60      0.03      0.05      1552
           4       0.46      0.04      0.07      2121
           5       0.56      0.05      0.09      1848
           6       0.95      0.22      0.36       833

   micro avg       0.72      0.46      0.56     20480
   macro avg       0.62      0.24      0.28     20480
weighted avg       0.64      0.46      0.46     20480
 samples avg       0.73      0.52      0.56     20480

train report               precision    recall  f1-score   support

           0       0.99      1.00      0.99     30797
           1       1.00      0.84      0.91      7272
           2       0.99      0.97      0.98     18328
           3       1.00      0.84      0.91      6199
           4       1.00      0.87      0.93      8430

[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.1s finished


In [ ]:
#doesn't know n_jobs
#won't take multilabel by default
#doesn't seem to be working

#svc = SVC(probability=True, verbose = True)
#svc.fit(X_train_one, y_train_one)

In [ ]:
#svc.score(X_test,y_test)

In [45]:
lr = LogisticRegression()
lr.fit(X_train_one, y_train_one)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
lr.score(X_test_one,y_test_one)

0.6396440445922159